In [1]:
import time
import urllib
import pnwkit
import pandas as pd
import sqlalchemy as sq
from sqlalchemy import create_engine
from urllib.parse import quote_plus

In [2]:
#DB Connection
conFormat = urllib.parse.quote_plus(
    'Data Source Name=Franklin;' 
    'Driver={SQL Server};' 
    'Server=FRANKLIN;' 
    'Database=PnWDB;' 
    'Trusted_connection=yes;'
)
#create engine
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conFormat))

# Extract

In [3]:
#Query GraphQL API using PnwKit-Py Wrapper
#Insert API Key here
kit = pnwkit.QueryKit("5992d973dadb75")

#Query first num of results at once
#Retrieves only fields attributes specified in .query() method
nation_query = kit.query(
    'nations', {'first': 500}, 'id', 'nation_name', 'leader_name', 'num_cities', 'date', 'color', 'last_active',
    'score', 'alliance_id', 'gross_domestic_product', 'gross_national_income', 'population', 'projects',
    'turns_since_last_city', 'turns_since_last_project', 'wars_won', 'wars_lost', 'soldiers', 'tanks', 'aircraft',
    'ships', 'missiles', 'nukes'
)
#Query alliance field and attributes
alliance_query = kit.query(
    'alliances', {'first': 100}, 'id', 'name', 'score', 'color', 'date', 'nations { id }' 
)
#Query treaty field and attributes
treaty_query = kit.query(
    'treaties', {'first': 500}, 'id', 'treaty_type', 'date', 'alliance1_id', 'alliance2_id', 'turns_left'
)
#Query city field and attributes
city_query = kit.query(
    'cities', {'first': 500}, 'id', 'name', 'nation { nation_name }', 'nation_id', 'date', 'infrastructure',
    'land', 'powered'
)

#Paginate queries for all available results in each field
nations = nation_query.paginate("nations")
alliances = alliance_query.paginate("alliances")
treaties = treaty_query.paginate("treaties")
cities = city_query.paginate("cities")

#Initialize empty dictionaries that will later be put converted into Dataframes
#Also set column order here
nation_data = {
    'id': [], 'nation_name': [], 'leader_name': [], 'num_cities': [], 'date': [], 'color': [], 'score': [],
    'alliance_id': [], 'population': [], 'gross_domestic_product': [], 'gross_national_income': [], 'projects': [],
    'turns_since_last_city': [], 'turns_since_last_project': [], 'wars_won': [], 'wars_lost': [], 'last_active': []
}
military_data = {
    'id': [], 'alliance_id': [], 'soldiers': [], 'tanks': [], 'aircraft': [], 'ships': [], 'missiles': [],
    'nukes': []
}
alliance_data = {
    'id': [], 'name': [], 'members': [], 'date': [], 'color': [], 'score': []
}
treaty_data = {
    'id': [], 'treaty_type': [], 'alliance1_id': [], 'alliance2_id': [], 'date': [], 'turns_left': []
}
city_data = {
    'id': [], 'name': [], 'nation': [], 'nation_id': [], 'date': [], 'infrastructure': [], 'land': []
}

In [4]:
#Convert paginator objects to dictionaries
#Iterate through paginator object: nations
for nation in nations:
    #iterate through empty dict and append values
    for key in nation_data:
        nation_data[str(key)].append(nation.__getattribute__(key))
    for key in military_data:
        military_data[str(key)].append(nation.__getattribute__(key))

#Iterate through paginator object: alliances
for alliance in alliances:
    #iterate through empty dict and append values
    for key in alliance_data:
        if key != 'members':
            alliance_data[str(key)].append(alliance.__getattribute__(key))
        #Special case where key != object atrribute
        else:
            alliance_data['members'].append(len(alliance.nations))

#Iterate through paginator object: treaties
for treaty in treaties:
    #iterate through empty dict and append values
    for key in treaty_data:
        treaty_data[str(key)].append(treaty.__getattribute__(key))

#Iterate through paginator object: cities
for city in cities:
    for key in city_data:
        #iterate through empty dict and append values
        if key != 'nation':
            city_data[str(key)].append(city.__getattribute__(key))
        #Special case where key != object atrribute
        else:
            #nested attribute nation_name
            city_data['nation'].append(city.nation.nation_name)

# Transform

In [11]:
#create DF from dictionaries
nations_df = pd.DataFrame(nation_data)
#change dtypes to appropriate datetime format
nations_df['date'] = pd.to_datetime(nations_df.date).dt.tz_localize(None)
nations_df['last_active'] = pd.to_datetime(nations_df.last_active).dt.tz_localize(None)
#change column headers to more approprate names
nations_df.rename(columns={'id': 'nation_id', 'num_cities': 'cities', 'date': 'founded', 'color': 'color_bloc',
                           'score': 'nation_score', 'gross_domestic_product': 'gdp', 'gross_national_income': 'gni', 
                           'projects':'national_projects'}, inplace=True)
nations_df.head()

,nation_id,nation_name,leader_name,cities,founded,color_bloc,nation_score,alliance_id,population,gdp,gni,national_projects,turns_since_last_city,turns_since_last_project,wars_won,wars_lost,last_active
0,5270,Belleroph,Callisto,39,2014-08-05 00:09:31,pink,7266.25,10523,15936475,21584959783,16495881175,18,537,623,32,95,2023-03-30 01:42:38
1,582,Nightsilver Woods,Luna,23,2014-08-05 00:09:35,gray,1840.11,615,676,112203203,-451757580,9,8207,8207,2,112,2022-02-26 21:12:20
2,439,Howell,Malal,31,2014-08-05 00:10:03,brown,5585.00,6088,11103358,15780012957,13282654045,19,1765,2636,190,257,2023-03-30 07:12:58
3,300,BEPU,Aenir,45,2014-08-05 00:10:40,green,9275.50,3339,20171683,29750549064,23064837275,29,2638,2505,162,132,2023-03-30 18:12:21
4,405,Gladius,Samwise,42,2014-08-05 00:10:41,green,8577.75,3339,18756477,26404570387,20934634860,28,288,2735,74,108,2023-03-30 02:38:11


In [6]:
#create DF from dictionaries
militaries_df = pd.DataFrame(military_data)
#change column headers to more approprate names
militaries_df.rename(columns={'id': 'nation_id'}, inplace=True)
militaries_df.head()

,nation_id,alliance_id,soldiers,tanks,aircraft,ships,missiles,nukes
0,5270,10523,0,19500,2925,0,0,0
1,582,615,0,0,0,0,0,0
2,439,6088,0,15500,2325,0,0,0
3,300,3339,180000,31500,2520,380,0,1
4,405,3339,165000,28750,2310,305,0,0


In [7]:
#create DF from dictionaries
alliances_df = pd.DataFrame(alliance_data)
#change dtypes to appropriate datetime format
alliances_df['date'] = pd.to_datetime(alliances_df.date).dt.tz_localize(None)
#change column headers to more approprate names
alliances_df.rename(columns={'id': 'alliance_id', 'name': 'alliance_name', 'date': 'founded',
                             'color': 'color_bloc', 'score': 'alliance_score'}, inplace=True)
alliances_df.head()

,alliance_id,alliance_name,members,founded,color_bloc,alliance_score
0,790,Rose,248,2014-11-04 23:13:16,pink,956286.63
1,1584,The Knights Radiant,155,2015-06-29 17:25:50,orange,592021.90
2,831,The Syndicate,95,2014-11-21 17:26:29,green,576237.69
3,1023,The Fighting Pacifists,145,2015-01-26 15:22:05,white,523897.36
4,5875,The Immortals,210,2019-08-04 19:30:43,yellow,521772.92


In [8]:
#create DF from dictionaries
treaties_df = pd.DataFrame(treaty_data)
#change dtypes to appropriate datetime format
treaties_df['date'] = pd.to_datetime(treaties_df.date).dt.tz_localize(None)
#change column headers to more approprate names
treaties_df.rename(columns={'id': 'treaty_id', 'alliance1_id':'sending_alliance_id', 'alliance2_id':'receiving_alliance_id'
                            , 'date': 'date_signed'}, inplace=True)
treaties_df.head()

,treaty_id,treaty_type,sending_alliance_id,receiving_alliance_id,date_signed,turns_left
0,13176,ODP,8526,4124,2021-05-14 22:03:09,92
1,13685,Protectorate,8654,4124,2021-06-04 11:44:31,92
2,14001,ODP,8709,8451,2021-06-30 03:36:36,118
3,15704,MDP,9292,6703,2021-12-11 05:50:06,262
4,15773,MDP,9292,7930,2021-12-11 05:53:13,112


In [12]:
#create DF from dictionaries
cities_df = pd.DataFrame(city_data)
#change dtypes to appropriate datetime format
cities_df['date'] = pd.to_datetime(cities_df.date).dt.tz_localize(None)
#change column headers to more approprate names
cities_df.rename(columns={'id': 'city_id', 'name': 'city_name', 'nation':'nation_name', 'date': 'date_built'}, 
                 inplace=True)
cities_df.head()

,city_id,city_name,nation_name,nation_id,date_built,infrastructure,land
0,10603,Flanaca,Belleroph,5270,2014-08-05,2750.0,4500.0
1,10604,Selemne,Nightsilver Woods,582,2014-08-05,0.2,3020.0
2,10605,Howell,Howell,439,2014-08-05,2400.0,5000.0
3,10614,in the city of,BEPU,300,2014-08-05,3000.0,5020.0
4,10615,Green Dragon,Gladius,405,2014-08-05,3000.0,4500.0


# Load into DB

In [26]:
#send tables into DB, preserving column datatypes
nations_df.to_sql('nations', con=engine, schema='dbo', if_exists='replace', index=False,
                  dtype={'nation_id': sq.types.INT(), 'nation_name': sq.types.VARCHAR(), 'leader_name': sq.types.VARCHAR(),
                         'cities': sq.types.INT(), 'founded': sq.types.DATETIME(), 'color_bloc': sq.types.VARCHAR(),
                         'nation_score': sq.types.DECIMAL(), 'alliance_id': sq.types.SMALLINT(),
                         'population': sq.types.INTEGER(), 'gdp': sq.types.DECIMAL(), 'gni': sq.types.DECIMAL(),
                         'national_projects': sq.types.SMALLINT(), 'turns_since_last_city': sq.types.SMALLINT(),
                         'turns_since_last_project': sq.types.SMALLINT(), 'wars_won': sq.types.SMALLINT(),
                         'wars_lost': sq.types.SMALLINT(), 'last_active': sq.types.DATETIME()})

militaries_df.to_sql('militaries', con=engine, schema='dbo', if_exists='replace', index=False,
                    dtype={'nation_id': sq.types.INT(), 'alliance_id': sq.types.SMALLINT(), 'soldiers': sq.types.INT(),
                           'tanks': sq.types.INT(), 'aircraft': sq.types.SMALLINT(), 'ships': sq.types.SMALLINT()})

alliances_df.to_sql('alliances', con=engine, schema='dbo', if_exists='replace', index=False,
                    dtype={'alliance_id': sq.types.SMALLINT(),'alliance_name': sq.types.VARCHAR(),
                           'members': sq.types.SMALLINT(), 'founded': sq.types.DATETIME(),'color_bloc': sq.types.VARCHAR(),
                           'alliance_score': sq.types.DECIMAL()})

treaties_df.to_sql('treaties', con=engine, schema='dbo', if_exists='replace', index=False,
                   dtype={'treaty_id': sq.types.INT(), 'treaty_type': sq.types.VARCHAR(),
                          'sending_alliance_id': sq.types.SMALLINT(), 'receiving_alliance_id': sq.types.SMALLINT(),
                          'date_signed': sq.types.DATETIME(), 'turns_left': sq.types.SMALLINT()})

cities_df.to_sql('cities', con=engine, schema='dbo', if_exists='replace', index=False,
                    dtype={'date_built': sq.types.Date()})

-1